In [ ]:
import pandas as pd

x1 = pd.ExcelFile('raw_data_for_phone.xlsx')
df = x1.parse('Sheet1')  # Replace 'Sheet1' with the name of your sheet if it's different
# ids = df['ID'].tolist()
print(df.columns)
addresses = df['address'].tolist()

print(len(addresses))
# print(addresses)

Index(['address', 'HouseNumber', 'CompassPoint', 'StreetName', 'Artery',
       'ZipCode'],
      dtype='object')
859068


In [ ]:
import re

error_list=[]

def process_data(address):
    if not isinstance(address, str):
      return ['', '', '', '', '']

    house_number_re = re.compile(r'^[\d-]+')
    compass_point_re = re.compile(r'(?<= )([NESW]|North|East|South|West)(?= )', re.IGNORECASE)
    street_name_re = re.compile(r'(?:(?<=[NESW] )|(?<= ))[\w\s-]+(?= (St|Street|Rd|Road|Ave|Avenue|Blvd|Boulevard|AV|PL|TERR|BR|NA|Broadway|NONE|DR|CT|PKWY|DRIVE|PARKWAY))', re.IGNORECASE)
    artery_re = re.compile(r'(?<= )(St|Street|Rd|Road|Ave|Avenue|Blvd|Boulevard|Av|Pl|TERR|BR|DR|CT|PKWY|DRIVE|Broadway|PARKWAY)(?= |$)', re.IGNORECASE)
    zip_code_re = re.compile(r'\b\d{5}\b')

    compass_dict = {
    's': 'South',
    'n': 'North',
    'e': 'East',
    'w': 'West',
    }

    artery_dict = {
    'st': 'Street',
    'av': 'Avenue',
    'ave': 'Avenue',
    'rd': 'Road',
    'blvd':'Boulevard',
    'pl':'Plaza',
    'terr':'Terrance',
    'br':'Bridge',
    'dr':'Drive',
    'ct':'Court'
    }


    address = re.sub(r'United States', '', address, flags=re.IGNORECASE)
    address = re.sub(r'New York', '', address, 1, flags=re.IGNORECASE) if address.lower().count('new york') > 1 else address

    house_number = house_number_re.search(address)
    house_number = house_number.group() if house_number else ''

    compass_point = compass_point_re.search(address)
    if compass_point:

      compass_point = compass_point.group()
      oc = compass_point
      compass_point = compass_dict.get(compass_point.lower(),compass_point)
    else:
      compass_point= ''

    street_name = street_name_re.search(address)
    if street_name:
        street_name = street_name.group().strip()
        # If the street name starts with a compass point, remove it
        if compass_point and street_name.startswith(oc):
            street_name = street_name[len(oc)+1:].strip() # +1 added to handle the space after compass point.
        else:
            street_name=street_name
    else:
        street_name = ''


    artery = artery_re.search(address)
    if artery:
        artery = artery.group()
        artery = artery_dict.get(artery.lower(), artery)
    else:
        artery = ''


    zip_code = zip_code_re.search(address)
    zip_code = zip_code.group() if zip_code else ''

    return [house_number, compass_point, street_name, artery, zip_code]


formatted_addresses = [process_data(address) for address in addresses]





In [ ]:
df_new = pd.DataFrame(formatted_addresses, columns=['HouseNumber', 'CompassPoint', 'StreetName', 'Artery', 'ZipCode'])


# concatenate the original DataFrame with the new one
df_final = pd.concat([df,df_new], axis=1)


# write the final DataFrame to a new Excel file
df_final.to_excel('splitaddress.xlsx', index=False)